#  Extract dataset and attribute definitions
##  from supportPrim - STATA files

In [65]:
# auto sugestion with tab
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

In [66]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'/lhome/amardj/ISM/python_code/es_scripts/pilot_scripts'

### Define the date format to be appended with file names

#### System defined current date

In [67]:
import datetime as datetime
sys_date = datetime.datetime.today().strftime( '%Y%m%d')
sys_date = f'{sys_date}' 
sys_date

'20201026'

In [68]:
#sys_date = '20201019'

### Location of raw data & generated data

In [69]:
data_dir = f'./data/{sys_date}/'
data_dir

'./data/20201026/'

In [70]:
index_name = 'id'

## Import stata files 'xxx.dta' for creating python pandas DataFrames

In [71]:
df_labeled = pd.read_stata(f'{data_dir}new_baseline_patient.dta') #, index_col=index_name)
df_labeled.shape # (*, 353)

(3, 350)

In [72]:
df_labeled.body_main_1

0    NaN
1    NaN
2    NaN
Name: body_main_1, dtype: category
Categories (0, object): []

In [73]:
assert df_labeled.employ_1.dtype.name == 'category' 
assert df_labeled.employ_1.dtype.name != 'object' 

df_labeled.employ_1.unique()

[working or other]
Categories (1, object): [working or other]

## Remove '_merege' cloumn form both the dataframes if any

In [74]:
drop_col_name = '_merge'
if drop_col_name in df_labeled.columns:
    df_labeled.drop( columns=[drop_col_name], axis=1, inplace=True)
    print(' The DataFrame contains column named as : ',drop_col_name, ' - Action : Removed')
else :
    print(' The DataFrame does not contain any column named as : "',drop_col_name,'"')

 The DataFrame does not contain any column named as : " _merge "


## Sort the dataframe column names - alfabetical order

In [75]:
df_labeled = df_labeled.reindex( sorted(df_labeled.columns), axis=1)
print(' Shape of old Dataframe : ', df_labeled.shape)  # (506, 292)
features_labels = df_labeled.columns

# Shape of old Dataframe :  (175, 353)

 Shape of old Dataframe :  (3, 350)


In [76]:
df_labeled.head()

,activity_1,age_1,back_nevro_pow_1,back_nevro_reflex_1,back_nevro_sens_1,back_slr_1,birth_year,bmi_1,body_main_1,childhood_1,...,treat_setting_self_some_3,treatmentfactor_patient_3,walk_aid_1,weight_1,work_ability_1,work_ability_3,work_ability_w2,work_ability_w4,work_ability_w8,work_type_1
0,quite reduced,59.0,NaN,NaN,NaN,NaN,1961,24.977043,NaN,very good,...,NaN,,no walkaid,68.0,7,NaN,NaN,NaN,NaN,work demands much walking and lifting
1,slightly reduced,63.0,NaN,NaN,NaN,NaN,1957,NaN,NaN,good,...,NaN,,no walkaid,NaN,6,NaN,NaN,NaN,NaN,mostly seated
2,quite reduced,30.0,NaN,NaN,NaN,NaN,1990,19.151192,NaN,good,...,NaN,,no walkaid,56.0,5,NaN,NaN,NaN,NaN,mostly seated


In [77]:
def replace_all_with( old, new, tempdf=df_labeled):
    temp = tempdf.replace({old: new}, regex=True)
    return temp

In [78]:
df_labeled.physio_id.head()

0    support27
1    support27
2    support42
Name: physio_id, dtype: object

In [79]:
df_labeled.treat_desc1_3.tolist()

['', '', '']

In [80]:
df_labeled['physio_id'] = df_labeled['physio_id'].replace(' ', '_', regex=True).str.lower()
df_labeled['clinic_id'] = df_labeled['physio_id']

In [81]:
df_labeled.physio_id

0    support27
1    support27
2    support42
Name: physio_id, dtype: object

In [82]:
cols = ['physio_id', 'journal_id', 'birth_year']
df_labeled['patient_id'] = df_labeled[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
df_labeled['patient_id'].tolist()

['support27_1005_1961', 'support27_1006_1957', 'support42_3002_1990']

In [83]:
df_labeled.index.name = 'id'
df_labeled.head()

,activity_1,age_1,back_nevro_pow_1,back_nevro_reflex_1,back_nevro_sens_1,back_slr_1,birth_year,bmi_1,body_main_1,childhood_1,...,walk_aid_1,weight_1,work_ability_1,work_ability_3,work_ability_w2,work_ability_w4,work_ability_w8,work_type_1,clinic_id,patient_id
id,,,,,,,,,,,,,,,,,,,,,
0,quite reduced,59.0,NaN,NaN,NaN,NaN,1961,24.977043,NaN,very good,...,no walkaid,68.0,7,NaN,NaN,NaN,NaN,work demands much walking and lifting,support27,support27_1005_1961
1,slightly reduced,63.0,NaN,NaN,NaN,NaN,1957,NaN,NaN,good,...,no walkaid,NaN,6,NaN,NaN,NaN,NaN,mostly seated,support27,support27_1006_1957
2,quite reduced,30.0,NaN,NaN,NaN,NaN,1990,19.151192,NaN,good,...,no walkaid,56.0,5,NaN,NaN,NaN,NaN,mostly seated,support42,support42_3002_1990


# Write the complete updated dataset to an excel sheet
## for: labeled and numeric dataset

In [84]:
df_labeled.to_csv( f'{data_dir}1_sp_dataset_labeled.csv', index=True)

In [85]:
print(df_labeled.columns.tolist())

['activity_1', 'age_1', 'back_nevro_pow_1', 'back_nevro_reflex_1', 'back_nevro_sens_1', 'back_slr_1', 'birth_year', 'bmi_1', 'body_main_1', 'childhood_1', 'como_asthma_1', 'como_cancer_1', 'como_count_1', 'como_dementhia_1', 'como_diabetes_1', 'como_fatigue_1', 'como_headache_1', 'como_heart_1', 'como_neuro_1', 'como_osteoporosis_1', 'como_other_1', 'como_other_spec_1', 'como_psych_1', 'como_ra_1', 'como_stomach_pain_1', 'compliance_3', 'consultation_3', 'date_baseline_patient', 'diagnose_1', 'discussed_employer_3', 'discussed_family_3', 'discussed_nav_3', 'discussed_occupational_3', 'education_1', 'education_othersp_1', 'employ_1', 'employ_assess_1', 'employ_assess_3', 'employ_disability_1', 'employ_disability_3', 'employ_disabilityp_1', 'employ_disabilityp_3', 'employ_paid_1', 'employ_paid_3', 'employ_paidp_1', 'employ_paidp_3', 'employ_parentleave_1', 'employ_parentleave_3', 'employ_retired_1', 'employ_retired_3', 'employ_sick_1', 'employ_sick_3', 'employ_sickp_1', 'employ_sickp_3',

## Do not delete the below lines of csv reading code.

In [86]:
df_labeled = pd.read_csv(f'{data_dir}1_sp_dataset_labeled.csv', index_col='id')
#df_numeric = pd.read_csv(file_numeric, index_col='id')

In [87]:
assert df_labeled.employ_1.dtype.name != 'category' 
assert df_labeled.employ_1.dtype.name == 'object' 

df_labeled.employ_1.unique()

array([' working or other'], dtype=object)

In [88]:
outcome_list = [
'out01',
#'out02'
]

In [89]:
df_labeled[outcome_list][:10]

,out01
id,
0,NaN
1,NaN
2,NaN


In [90]:
df = df_labeled.copy( deep=True)

In [91]:
df.body_main_1.unique()

array([nan])

In [92]:
df.shape

(3, 352)

In [93]:
df.education_1.head()

id
0        up to 4 years higher education
1                           high school
2    more than 4 years higher education
Name: education_1, dtype: object

In [94]:
df.employ_1.head()

id
0     working or other
1     working or other
2     working or other
Name: employ_1, dtype: object

# 
### Rounding of the float variables up to 2 decimal places

In [95]:
round_float_list = ['hscl_score_1',
                    #'ndi_total_1',  
                    #'odi_total_1',  
                    #'spadi_total_1',
                    #'outcome_allpercent',
                    
                    'hscl_score_3',
                    #'ndi_total_3',  
                    #'odi_total_3',
                    #'spadi_total_3'
                   ]

for col_name in round_float_list:
    print(col_name)
    df[col_name] = df[col_name].round(2)
    df_labeled[col_name] = df[col_name].round(2)
    #df_numeric[col_name] = df_numeric[col_name].round(2)
    
# df_labeled.hscl10_mean_1 = df_labeled.hscl10_mean_1.round(2)
# df_labeled.ndi_total_1   = df_labeled.ndi_total_1.round(2)
# df_labeled.odi_total_1   = df_labeled.odi_total_1.round(2)
# df_labeled.spadi_total_1 = df_labeled.spadi_total_1.round(2)
# 
# df_numeric.hscl10_mean_3 = df_numeric.hscl10_mean_3.round(2)
# df_numeric.spadi_total_3 = df_numeric.spadi_total_3.round(2)
# df_numeric.ndi_total_3   = df_numeric.ndi_total_3.round(2)
# df_numeric.odi_total_3   = df_numeric.odi_total_3.round(2)

hscl_score_1
hscl_score_3


In [96]:
# for testing
#print(df_labeled[ round_float_list][:5]);
#print(df_numeric[ round_float_list][:5]);

## Drop all empty columns

#### Issues with complete empty columns ????

## Function for removing all leading and trailing spaces in a dataframe

In [97]:
def remove_leading_trailing_spaces(tempdf=df):
    df_obj = tempdf.select_dtypes(['object'])
    tempdf[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    return tempdf

## Function to repalce all characters of a string in a dataframe

In [98]:
def replace_all_with( old, new, tempdf=df):
    temp = tempdf.replace({old: new}, regex=True)
    return temp

In [99]:
# Must throw exception
# df_numeric.employment_1 = replace_all_with( 'Working,', 'Working ', df_numeric.employment_1)
# df_numeric.employment_1.unique()

## Replace all special char

### 1. '>' is replaced with 'GT' (Greater Than) and '<' is replaced with 'LT' (Lower/Less Than)

In [100]:
#df_labeled.outcome_percent01.unique()

In [101]:
df = replace_all_with('>', 'GT ', df)
df = replace_all_with('≥', 'GT_EQ ', df)
df = replace_all_with('<', 'LT ', df)

df_labeled = replace_all_with('>', 'GT ', df_labeled)
df_labeled = replace_all_with('≥', 'GT_EQ ', df_labeled)
df_labeled = replace_all_with('<', 'LT ', df_labeled)

#df_numeric = replace_all_with('>', 'GT ', df_numeric)
#df_numeric = replace_all_with('<', 'LT ', df_numeric)

In [102]:
#df_labeled.outcome_percent01.unique()

### 2. '-'  to  ' - ' for readablity
### 3. ';'  to  ':' (colon) to compenseate the next statement (4.) 
### 4. ','  to  ';' (semicolon) since they represent multiple values for a given attribute, and they will result in a 'csv' file 

In [103]:
df = replace_all_with( '-', ' - ', df)
#df = replace_all_with( ';', ' : ', df)
#df = replace_all_with( ',', ' ; ', df)

df_labeled = replace_all_with( '-', ' - ', df_labeled)
#df_labeled = replace_all_with( ';', ' : ', df_labeled)
#df_labeled = replace_all_with( ',', ' ; ', df_labeled)

#df_numeric = replace_all_with( '-', ' - ', df_numeric)
#df_numeric = replace_all_with( ';', ' : ', df_numeric)
#df_numeric = replace_all_with( ',', ' ; ', df_numeric)

### 5. '%'  to  ' percent' to resolve the special char ambiguity

In [104]:
df = replace_all_with( '%', ' percent', df)
df_labeled = replace_all_with( '%', ' percent', df_labeled)
#df_numeric = replace_all_with( '%', ' percent', df_numeric)

In [105]:
df.body_main_1.unique()

array([nan])

In [106]:
df = replace_all_with( '/', '_', df)
df_labeled = replace_all_with( '/', '_', df_labeled)

In [107]:
df.body_main_1.unique() # The data type should not be cateorical type for python

array([nan])

In [108]:
#df.body_main_1.cat.rename_categories({'multisite/complex': 'multisite_complex'}, inplace=True)
#df_labeled.body_main_1.cat.rename_categories({'multisite/complex': 'multisite_complex'}, inplace=True)

In [109]:
df.body_main_1.unique()

array([nan])

### 6. multiple spaces in the text to resolve the ambiguity in categorical values for myCBR

In [110]:
df = replace_all_with( '   ', ' ', df)
df = replace_all_with( '  ', ' ',  df)

df_labeled = replace_all_with( '   ', ' ', df_labeled)
df_labeled = replace_all_with( '  ', ' ',  df_labeled)

#df_numeric = replace_all_with( '   ', ' ', df_numeric)
#df_numeric = replace_all_with( '  ', ' ', df_numeric)

In [111]:
#df_labeled.treat_base[:50]

### 7. The attribute was originally like ''No, off and on', where  ',' was replaced to  ';'. But  ';' is the representation for multiple value thus we  need to convert  ';' to  ':' for its intended interpretation.

In [112]:
df.pain_continuous_1.head()

id
0     no
1     no
2    yes
Name: pain_continuous_1, dtype: object

In [113]:
df_labeled.medic_number_1.unique()

array(['0', '1-4'], dtype=object)

In [114]:
df.medic_number_1 = df.medic_number_1.replace({'-': 'to'}, regex=True)
df_labeled.medic_number_1 = df_labeled.medic_number_1.replace({'-': 'to'}, regex=True)
print(df.medic_number_1.unique())

#df.medic_number_3 = df.medic_number_3.replace({'-': 'to'}, regex=True)
#df_labeled.medic_number_3 = df_labeled.medic_number_3.replace({'-': 'to'}, regex=True)
#print(df.medic_number_3.unique())

#df.medic_number_1.cat.rename_categories({'1-4': '1 to 4'}, inplace=True)
#df_labeled.medic_number_1.cat.rename_categories({'1-4': '1 to 4'}, inplace=True)
#print(df.medic_number_1.unique())
#
#df.medic_number_3.cat.rename_categories({'1-4': '1 to 4'}, inplace=True)
#df_labeled.medic_number_3.cat.rename_categories({'1-4': '1 to 4'}, inplace=True)
#print(df.medic_number_3.unique())
##df.treat_int_3 = df.treat_int_3.replace({'-': 'to'}, regex=True)
##df.treat_int_3.unique()
#
##df_labeled.treat_int_3 = df_labeled.treat_int_3.replace({'-': 'to'}, regex=True)
##df_labeled.treat_int_3.unique()

['0' '1to4']


In [115]:
if( df.columns.contains('priokey')):
    df.priokey.unique()
    df.priokey = df.priokey.replace({';': '-'}, regex=True)

    print(df.priokey.unique())

/lhome/amardj/anaconda3/envs/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  """Entry point for launching an IPython kernel.


In [116]:
df.employ_1.unique()

array([' working or other'], dtype=object)

In [117]:
df.shape  # (*, 355)

(3, 352)

In [118]:
df.como_other_spec_1.unique().tolist()

[nan]

In [119]:
if not df.como_other_spec_1.dtype in [int, float]:
    df.como_other_spec_1.replace({' _ ':'-'}, regex=True, inplace=True)

In [120]:
if not df.treat_goal_1.dtype in [int, float]:
    df.treat_goal_1.replace({' _ ':'-'}, regex=True, inplace=True)

In [121]:
df.treat_goal_1.tolist()

[nan, nan, nan]

In [122]:
#df.outcome_percent01 = df.outcome_percent01.replace({'â¥50': 'GT_EQ 50'}, regex=True)
#df.outcome_percent01.unique()

In [123]:
#df[['outcome_allpercent','outcome_percent','outcome_percent01']][:100]

In [124]:
#df_labeled.outcome_percent.unique()

In [125]:
#df_labeled.outcome_percent = df_labeled.outcome_percent.replace({'-': 'to'}, regex=True)
#df_labeled.outcome_percent = df_labeled.outcome_percent.replace({'change/worse': 'change_worse'}, regex=True)
#df_labeled.outcome_percent.unique()

## Remove leading or trailing spaces

In [126]:
df = remove_leading_trailing_spaces( tempdf=df)

df_labeled = remove_leading_trailing_spaces( tempdf=df_labeled)

#df_numeric = remove_leading_trailing_spaces( tempdf=df_numeric)

In [127]:
df.employ_1.unique()

array(['working or other'], dtype=object)

## Write the feature engineered dataframe to a "csv" file for myCBR analysis

In [128]:
df.to_csv( f'{data_dir}2_sp_dataset_clean.csv', index=True)